In [5]:
import pandas as pd
import zipfile
from pyspark.sql import SparkSession
from azure.storage.blob import BlobServiceClient

In [2]:
def spark_session():
    spark = SparkSession.builder.appName("app").getOrCreate()
    spark.conf.set("fs.azure.account.key.cahospitalstorage.blob.core.windows.net", \
    "w3EYeDH//L+J0uscsn3BYDatbFCcB7CUXK4MfOSNmwc16v3WAQ7EPcw2IpchUrelLVuowjfugmv99e0Os8J5Sw==")
    return spark

def connect_blob():
    storage_conn_string = "DefaultEndpointsProtocol=https;AccountName=cahospitalstorage;AccountKey=w3EYeDH//L+J0uscsn3BYDatbFCcB7CUXK4MfOSNmwc16v3WAQ7EPcw2IpchUrelLVuowjfugmv99e0Os8J5Sw==;EndpointSuffix=core.windows.net"
    blob_service_client = BlobServiceClient.from_connection_string(storage_conn_string)
    return blob_service_client 

def get_blob_data(blob_service_client, container, blob_location):
    container_client = blob_service_client.get_container_client(container)
    blob_data = container_client.get_blob_client(blob_location)
    if not blob_data.exists():
        raise FileNotFoundError('File not found in given container.')
    return container_client, blob_data
    
def blob_to_temp(filetype, blob_data):
    if not blob_data.exists():
        raise OSError('File not found in given container.')
        
    if filetype == 'excel':
        temp_name = "temp.xlsx"
    elif filetype == 'csv':
        temp_name = "temp.txt"
    else:
        raise ValueError('Incorrect filetype entered. Only excel and csv files accepted')
    with open(temp_name, "wb") as temp:
        data = blob_data.download_blob()
        data.readinto(temp)
        
def read_blob_data(filetype, sheetname=0, dtype=None, delim=None, skiprows=None, usecols=None, columns=None):
    if filetype == 'excel':
        return pd.read_excel("temp.xlsx", sheet_name=sheetname, skiprows=skiprows, names=columns, engine="openpyxl")
    elif filetype == 'csv':
        return pd.read_csv("temp.txt", dtype='unicode', delimiter=delim, skiprows=skiprows, names=columns)
    else:
        raise ValueError('Incorrect filetype entered. Only excel and csv files accepted')
        
def export_to_blob(output, blob_name, container_client):
    for file in os.listdir(output):
        try:
            with open("%s/%s" % (output, file), "rb") as data:
                container_client.upload_blob(name='%s/%s' % (blob_name, file), data=data, overwrite=True)
                data.close()
        except Exception as e:
            print(e)

In [4]:
def sheet_names(element):
    file = element[0]
    container_client = element[1]
    blob_data = container_client.get_blob_client(file)
    blob = blob_data.download_blob().readall()
    try:
        df = pd.read_excel(blob, sheet_name=None, engine="openpyxl")
        return (file, list(df.keys()))
    except (OSError, zipfile.BadZipFile): 
        return (None, None)

def read_files(row):
    file = row[0]
    sheet = row[1]
    container_client = row[2]
    match_string = 'Evaluation & Management Services (CPT Codes 99201-99499)'
  
    blob_data = container_client.get_blob_client(file)
    blob = blob_data.download_blob().readall()
    try:
        df = pd.read_excel(blob, sheet_name=sheet, engine="openpyxl")
        df_string = df.to_string()
        if match_string in df_string:
            return (file, sheet)
        return (None, None)
    except OSError:
        return (None, None)
